Approach:
    
I have used Generative LSTM based language model due to the unavailability of GPU or higher CPU.

In [ ]:
# dependencies

import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
import datasets

In [ ]:
max_len = 50
data = datasets.readCornellData("data/cornell", max_len = max_len)

In [ ]:
import pickle
with open('cornell_data_%d'%max_len, 'wb') as file:
    pickle.dump(data, file)

In [16]:
import pickle
with open('cornell_data_50', 'rb') as file:
    data = pickle.load(file)

In [17]:
data[:10]

[('gosh if only we could find kat a boyfriend', 'let me see what i can do'),
 ('cesc ma tete this is my head', 'right see youre ready for the quiz'),
 ('thats because its such a nice one', 'forget french'),
 ('there', 'where'),
 ('you have my word as a gentleman', 'youre sweet'),
 ('hi', 'looks like things worked out tonight huh'),
 ('you know chastity', 'i believe we share an art instructor'),
 ('have fun tonight', 'tons'),
 ('i was', 'you never wanted to go out with me did you'),
 ('well no', 'then thats all you had to say')]

In [18]:
print (len(data))

101349


In [ ]:
type(data)

In [19]:
word2idx = {'A' : 1,  'B' : 2}

word = 'A B C D'

for word in word.split(' '):
    if word not in word2idx:
        print ('not there')

not there
not there


In [20]:
# filter out sentences above a max threshold for both the question and response.

MAX_LEN = 10

def remove_pairs_above_max_thresh(pairs, MAX_LEN):
    
    if ((len(pairs[0].split(' ')) <= MAX_LEN) & 
        (len(pairs[1].split(' ')) <= MAX_LEN)):
        return True
    else:
        return False    

In [21]:
new_data = [p for p in data if remove_pairs_above_max_thresh(p, MAX_LEN)]

In [22]:
new_data[:10]

[('gosh if only we could find kat a boyfriend', 'let me see what i can do'),
 ('cesc ma tete this is my head', 'right see youre ready for the quiz'),
 ('thats because its such a nice one', 'forget french'),
 ('there', 'where'),
 ('you have my word as a gentleman', 'youre sweet'),
 ('hi', 'looks like things worked out tonight huh'),
 ('you know chastity', 'i believe we share an art instructor'),
 ('have fun tonight', 'tons'),
 ('i was', 'you never wanted to go out with me did you'),
 ('well no', 'then thats all you had to say')]

In [23]:
print (len(new_data))

97104


In [ ]:
data[0][0] +  ' '  + data[0][1]

In [24]:
(data[9][0] +  ' '  + data[9][1]).split(" ")

['well', 'no', 'then', 'thats', 'all', 'you', 'had', 'to', 'say']

In [25]:
# need to pack pair query, response into one corpus of words
from collections import Counter

total_vocab = Counter()

combined = []
for j in range(len(data)):
    for word in (data[j][0] + ' ' + data[j][0]).split():
        total_vocab[word] += 1

In [26]:
total_vocab.most_common()[-10:]

[('hay', 2),
 ('conti', 2),
 ('nental', 2),
 ('mmmmmmmmm', 2),
 ('fuchsmachen', 2),
 ('sweetheartis', 2),
 ('schwanzstucker', 2),
 ('tenthirty', 2),
 ('grrrhmmnnnjkjmmmnn', 2),
 ('noggs', 2)]

In [27]:
# total size of the vocab

vocab = set(total_vocab.keys())
print ("Vocab size is: {:,}".format(len(vocab)))

Vocab size is: 21,800


In [28]:
# let's drop any word which has frequency below a min count
# in our case, let's use 5 for example

keep_words = []
drop_words = []
min_cnt = 3

for k, v in total_vocab.items():
    if v < min_cnt:
        drop_words.append(k)
    elif v >= min_cnt:
        keep_words.append(k)
        
print (len(drop_words))
print (len(keep_words))

11086
10714


In [29]:
drop_words[:10]

['cesc',
 'tete',
 'unbalanced',
 '9th',
 'perish',
 'tutor',
 'shrew',
 'biancas',
 'smokers',
 'assail']

In [30]:
# adding special words to the keep_words list

keep_words[0] = "PAD"
keep_words[1] = "SOS"
keep_words[2] = "EOS"

keep_words[:10]

['PAD', 'SOS', 'EOS', 'we', 'could', 'find', 'kat', 'a', 'boyfriend', 'ma']

In [31]:
word2idx = {}
idx2word = {}

for i, word in enumerate(keep_words):
    word2idx[word] = i
    idx2word[i] = word

In [32]:
sample = {k: word2idx[k] for k in list(word2idx)[:16]}
sample

{'PAD': 0,
 'SOS': 1,
 'EOS': 2,
 'we': 3,
 'could': 4,
 'find': 5,
 'kat': 6,
 'a': 7,
 'boyfriend': 8,
 'ma': 9,
 'this': 10,
 'is': 11,
 'my': 12,
 'head': 13,
 'thats': 14,
 'because': 15}

In [45]:
# let's try to filter out pairs that do not contain the words in keep words

print ("The number of Question Answer pairs in new_data is: {:,}".format(len(new_data)))
new_data2 = []

for p in range(len(new_data)):
    
    input_sen = new_data[p][0]
    output_sen = new_data[p][1]
    
    keep_input = True
    keep_output = True
    
    for word in input_sen.split():
        
        if word not in keep_words:
            keep_input = False
            break
            
    for word in output_sen.split():
        
        if word not in keep_words:
            keep_output = False
            break
            
    if keep_input and keep_output:        
        new_data2.append((input_sen, output_sen))
        
print ("Trummed from {:,} pairs to {:,}, {:.2f} of total".format(len(new_data), len(new_data2),
                                                                len(new_data2)/len(new_data)))

The number of Question Answer pairs in new_data is: 97,104
Trummed from 97,104 pairs to 75,352, 0.78 of total


In [46]:
new_data2[:5]

[('thats because its such a nice one', 'forget french'),
 ('there', 'where'),
 ('you have my word as a gentleman', 'youre sweet'),
 ('hi', 'looks like things worked out tonight huh'),
 ('have fun tonight', 'tons')]

In [47]:
import itertools
list1 = [3, 5, 10]
list2 = [4, 9, 1, 3, 10]
list3 = [3, 3, 4, 5, 1, 6, 2, 1, 7, 10]

def test_ex(mylist, ml, val):
    if len(mylist) < ml:
        return mylist + [val] * (ml-len(mylist))
    else:
        return mylist
    
print (test_ex(list1, ml=10, val=0))
print (test_ex(list2, ml=10, val=0))
print (test_ex(list3, ml=10, val=0))

[3, 5, 10, 0, 0, 0, 0, 0, 0, 0]
[4, 9, 1, 3, 10, 0, 0, 0, 0, 0]
[3, 3, 4, 5, 1, 6, 2, 1, 7, 10]


In [48]:
# converting the input and output sentences into tensors and train with mini-batches
# data processing: since max_length = 10, any sentence shorter than 10 will need to be padded
# with zeros after EOS token


# padding function

def padding_func(mylist, ml, val):
    if len(mylist) < ml:
        return mylist + [val] * (ml-len(mylist))
    else:
        return mylist

# adding EOS 

max_len = 10
input_index = []
output_index = []  

pad_token = word2idx['PAD']
eos_token = word2idx['EOS']
    
for p in range(len(new_data2)):
    
    input_sen = new_data2[p][0]
    output_sen = new_data2[p][1]
    
    # convert both sentences to indexes using the above defined word2idx
    
    p_input = []
    p_output = []
    
    for word in input_sen.split():
        p_input.append(word2idx[word])        
        
    p_input.append(eos_token)
    p_input = padding_func(p_input, ml=max_len, val=pad_token)    
    input_index.append(p_input)
    
    
    for word in output_sen.split():
            p_output.append(word2idx[word])        

    p_output.append(eos_token)
    p_output = padding_func(p_output, ml=max_len, val=pad_token)    
    output_index.append(p_output)
    
    
for j in range(5):
    
    print (input_index[j])
    print (output_index[j])

[14, 15, 16, 17, 7, 18, 19, 2, 0, 0]
[241, 172, 2, 0, 0, 0, 0, 0, 0, 0]
[20, 2, 0, 0, 0, 0, 0, 0, 0, 0]
[61, 2, 0, 0, 0, 0, 0, 0, 0, 0]
[21, 22, 12, 23, 24, 7, 25, 2, 0, 0]
[200, 80, 2, 0, 0, 0, 0, 0, 0, 0]
[26, 2, 0, 0, 0, 0, 0, 0, 0, 0]
[1358, 87, 1183, 2820, 100, 30, 637, 2, 0, 0]
[22, 29, 30, 2, 0, 0, 0, 0, 0, 0]
[7514, 2, 0, 0, 0, 0, 0, 0, 0, 0]


In [62]:
print (len(input_index))
print (len(output_index))

75352
75352


In [58]:
def true_len(mylist):    
    return len(list(filter(lambda a: a != 0, mylist)))

In [59]:
for i in enumerate(input_index[:4]):
    print (i, true_len(i[1]))

(0, [14, 15, 16, 17, 7, 18, 19, 2, 0, 0]) 8
(1, [20, 2, 0, 0, 0, 0, 0, 0, 0, 0]) 2
(2, [21, 22, 12, 23, 24, 7, 25, 2, 0, 0]) 8
(3, [26, 2, 0, 0, 0, 0, 0, 0, 0, 0]) 2


In [61]:
# sorting the pairs by length of the questions

sorted_questions = []
sorted_answers = []

for length in range(1, max_len+1):
    
    for i in enumerate(input_index):
        if true_len(i[1]) == length:
            sorted_questions.append(input_index[i[0]])
            sorted_answers.append(output_index[i[0]])

print(len(sorted_questions))
print(len(sorted_answers))

72853
72853


In [64]:
for i in range(3):
    print(sorted_questions[i])
    print(sorted_answers[i])
    print ('*' * 30)

[20, 2, 0, 0, 0, 0, 0, 0, 0, 0]
[61, 2, 0, 0, 0, 0, 0, 0, 0, 0]
******************************
[26, 2, 0, 0, 0, 0, 0, 0, 0, 0]
[1358, 87, 1183, 2820, 100, 30, 637, 2, 0, 0]
******************************
[40, 2, 0, 0, 0, 0, 0, 0, 0, 0]
[21, 258, 86, 10, 8097, 2, 0, 0, 0, 0]
******************************


In [67]:
def model_inputs():
    
    
    input_data = tf.placeholder(tf.int32, [None, None], name = 'input')
    targets = tf.placeholder(tf.int32, [None, None], name = 'targets')
    lr = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.placeholder(tf.float32, name  ='keep_prob')

    return input_data, targets, lr, keep_prob

In [68]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the SOS to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['SOS']), ending], 1)

    return dec_input

In [70]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    '''Create the encoding layer'''
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell,
                                                   cell_bw = enc_cell,
                                                   sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)
    return enc_state

In [71]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size):
    '''Decode the training data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                     att_keys,
                                                                     att_vals,
                                                                     att_score_fn,
                                                                     att_construct_fn,
                                                                     name = "attn_dec_train")
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                              train_decoder_fn, 
                                                              dec_embed_input, 
                                                              sequence_length, 
                                                              scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    return output_fn(train_pred_drop)

In [72]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob, batch_size):
    '''Decode the prediction data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn =             tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    infer_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn, 
                                                                         encoder_state[0], 
                                                                         att_keys, 
                                                                         att_vals, 
                                                                         att_score_fn, 
                                                                         att_construct_fn, 
                                                                         dec_embeddings,
                                                                         start_of_sequence_id, 
                                                                         end_of_sequence_id, 
                                                                         maximum_length, 
                                                                         vocab_size, 
                                                                         name = "attn_dec_inf")
    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                                infer_decoder_fn, 
                                                                scope=decoding_scope)
    
    return infer_logits

In [73]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, batch_size):
    '''Create the decoding cell and input the parameters for the training and inference decoding layers'''
    
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
        
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, 
                                                                vocab_size, 
                                                                None, 
                                                                scope=decoding_scope,
                                                                weights_initializer = weights,
                                                                biases_initializer = biases)

        train_logits = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            sequence_length, 
                                            decoding_scope, 
                                            output_fn, 
                                            keep_prob, 
                                            batch_size)
        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            vocab_to_int['SOS'],
                                            vocab_to_int['EOS'], 
                                            sequence_length - 1, 
                                            vocab_size,
                                            decoding_scope, 
                                            output_fn, keep_prob, 
                                            batch_size)

    return train_logits, infer_logits


In [74]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, answers_vocab_size, 
                  questions_vocab_size, enc_embedding_size, dec_embedding_size, rnn_size, num_layers, 
                  questions_vocab_to_int):
    
    '''Use the previous functions to create the training and inference logits'''
    
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, 
                                                       answers_vocab_size+1, 
                                                       enc_embedding_size,
                                                       initializer = tf.random_uniform_initializer(0,1))
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length)

    dec_input = process_encoding_input(target_data, questions_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([questions_vocab_size+1, dec_embedding_size], 0, 1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    train_logits, infer_logits = decoding_layer(dec_embed_input, 
                                                dec_embeddings, 
                                                enc_state, 
                                                questions_vocab_size, 
                                                sequence_length, 
                                                rnn_size, 
                                                num_layers, 
                                                questions_vocab_to_int, 
                                                keep_prob, 
                                                batch_size)
    return train_logits, infer_logits

In [75]:
# Set the Hyperparameters
epochs = 100
batch_size = 128
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.005
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.75

In [80]:
import tensorflow as tf

In [79]:
# Reset the graph to ensure that it is ready for training
tf.reset_default_graph()
# Start the session
sess = tf.InteractiveSession()
    
# Load the model inputs    
input_data, targets, lr, keep_prob = model_inputs()

# Sequence length will be the max line length for each batch
sequence_length = tf.placeholder_with_default(max_line_length, None, name='sequence_length')

# Find the shape of the input data for sequence_loss
input_shape = tf.shape(input_data)

AttributeError: module 'tensorflow' has no attribute 'reset_default_graph'